# Evaluation of Multi-Agent approach using an orchestrator

In [5]:
import sys
import os
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates")
from dotenv import load_dotenv

sys.path.insert(0, os.path.abspath("../src"))
print(sys.path)

load_dotenv()
from approach_evaluator import ApproachEvaluator
from orchestrator_agent import create_sql_from_natural_text, visualize_graph
visualize_graph()

['/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/.venv/lib/python3.13/site-packages', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents', '/Users/i551965/Documents/dev/Uni/

In [6]:
import pandas as pd

data_dir = os.getenv("DATA_DIRECTORY")
sql_input_file = pd.read_csv("../sample/dev.csv")

# Config is just for logging purposes and does not affect the run
run_config = {
    "k_shot_prompting_creator_agent": 2,
    "k_shot_prompting_fixer_agent": 1,
    "k_shot_prompting_feedback_agent": 2,
    "k_shot_prompting_select_relevant_database": 0,
    "k_shot_prompting_select_relevant_tables": 2,
    "llm_orchestrator": "gpt4o",
    "llm_fixer": "gpt4o",
    "llm_feedback": "gpt4o",
    "llm_temperature": 0
}

evaluator = ApproachEvaluator(
    run_config=run_config,
    question_ids=sql_input_file["question_id"],
    approach=create_sql_from_natural_text
)
evaluator.evaluate()

{'Natural Language Prompt/ Original Question': 'Among the schools with the average score in Math over 560 in the SAT test, how many schools are directly charter-funded?', 'Syntactically Incorrect SQL Query': "SELECT COUNT(*) FROM schools WHERE AVG(MathScore) > 560 AND FundingType = 'charter';", 'Error Message': 'Error executing query: no such column: FundingType', 'Error Position': "FundingType = 'charter'", 'Specific Error': "FundingType is not a valid column in the 'schools' table.", 'Advise': "Check the schema of the 'schools' table to see if there is a column named 'FundingType'. If not, you may need to join the 'schools' table with another table that contains the funding information or use a different column that represents the funding type."}
Orchestrator Agent: An error occurred: 'feedback'
Error with final query: Database with name  not found.
Target query: SELECT COUNT(T2.`School Code`) FROM satscores AS T1 INNER JOIN frpm AS T2 ON T1.cds = T2.CDSCode WHERE T1.AvgScrMath > 560

{'question_id': [479,
  1057,
  1367,
  298,
  651,
  1145,
  1165,
  494,
  1126,
  1292,
  9,
  152,
  1254,
  609,
  823,
  467,
  489,
  1133,
  535,
  1019,
  1154,
  54,
  53,
  34,
  904,
  542,
  1341,
  279,
  491,
  1314,
  1161,
  268,
  841,
  1102,
  847,
  1300,
  1120,
  1441,
  920,
  648,
  113,
  686,
  1405,
  883,
  789,
  347,
  1211,
  1115,
  1045,
  227,
  1379,
  717,
  1495,
  1131,
  1124,
  692,
  1459,
  452,
  916,
  66,
  442,
  427,
  1044,
  549,
  1386,
  938,
  1357,
  477,
  713,
  1458,
  784,
  342,
  204,
  458,
  615,
  295,
  414,
  1236,
  935,
  776,
  817,
  353,
  1062,
  1501,
  159,
  435,
  892,
  748,
  192,
  678,
  1224,
  1177,
  31,
  512,
  202,
  1226,
  893,
  579,
  879,
  948],
 'predicted_query': ['',
  '',
  'SELECT AVG("FRPM Count (Ages 5-17)") FROM schools, frpm WHERE schools."Column name: School Name" = "Kacey Gibson" AND frpm."Column name: School Name" = "Kacey Gibson" AND schools."Column name: Charter" = "No" AND schools.

In [7]:
import pandas as pd
run = pd.read_csv("./results/results_full_run_291124/results.csv")

run.head()

,input_texts,predicted_query,predicted_database,target_queries,is_correct
0,Among the cards with converted mana cost highe...,SELECT COUNT(*) FROM cards WHERE convertedMana...,card_games,SELECT SUM(CASE WHEN T1.power LIKE '*' OR T1.p...,0
1,Calculate the average home team goal in the 20...,SELECT AVG(home_team_goal) FROM Match WHERE se...,european_football_2,SELECT CAST(SUM(t2.home_team_goal) AS REAL) / ...,1
2,Which college do most of the members go to?,SQL\r\n\r\nSELECT college\r\nFROM major\r\nWHE...,student_club,SELECT T2.college FROM member AS T1 INNER JOIN...,0
3,Calculate the percentage of molecules containi...,SELECT (CAST(COUNT(DISTINCT(molecule_id)) AS R...,toxicology,SELECT CAST(COUNT(CASE WHEN T1.element = 'h' A...,0
4,"Provide the related post title of ""How to tell...","SELECT ""Post Title"" FROM 'thrombosis_predictio...",thrombosis_prediction,SELECT T3.Title FROM postLinks AS T1 INNER JOI...,0


In [8]:
from approach_evaluator import valid_efficiency_score

ves = valid_efficiency_score(run)

print(f"Valid Efficiency Score: {ves}")

Valid Efficiency Score: 0.2046942101312174
